# GeekBrains, ML in Business
# Lesson 4 Homework

Ссылки:
* http://hyperopt.github.io/hyperopt/
* https://arxiv.org/pdf/1907.03947.pdf
* https://arxiv.org/pdf/1802.02301.pdf
* https://arxiv.org/list/stat.ML/recent
* https://scikit-learn.org/stable/modules/grid_search.html
* https://scikit-learn.org/stable/modules/compose.html

**Импорт библиотек**

In [1]:
import numpy as np
import pandas as pd

# from IPython.display import Image

# from sklearn.model_selection import train_test_split

# from sklift.metrics import uplift_at_k
# from sklift.viz import plot_uplift_preds
# from sklift.models import SoloModel, ClassTransformation, TwoModels

# from causalml.inference.tree import UpliftTreeClassifier, UpliftRandomForestClassifier
# from causalml.inference.tree import uplift_tree_string, uplift_tree_plot

# from catboost import CatBoostClassifier


# %matplotlib inline

## Задание 1
Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков).

### Решение Задания 1

## Задание 2
Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика).

### Решение Задания 2

## Задание 3
Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.

### Решение Задания 3

## Задание 4*
Провести подбор гиперпараметров лучшей модели по итогам 2-3.

### Решение Задания 4*

## Задание 5*
Еще раз провести оценку экономической эффективности.

### Решение Задания 5*